# 8.4.3	Den *fastText*-Vektorraum um unbekannte Wörter erweitern

### 01 - Heidegger-Buchtext als String laden und tokenisieren

#### a) Text als String laden und mit der Funktion *nltk.tokenize.word_tokenize* tokenisieren

In [1]:
import numpy as np
import joblib
from nltk.tokenize import word_tokenize
from os.path import join

path = r'..\Data'

with open(join(path, 'Heidegger_bereinigt.txt'), 'r', 
           encoding='latin-1') as doc:
     text = doc.read()
     text_words = word_tokenize(text)

text[:100], text_words[:10]

('denn offenbar seid ihr doch schon\nlange mit dem vertraut, was ihr eigentlich meint, wenn ihr den\naus',
 ['denn',
  'offenbar',
  'seid',
  'ihr',
  'doch',
  'schon',
  'lange',
  'mit',
  'dem',
  'vertraut'])

### 02 - Den *fastText*-Vektorraum laden
**Hinweis**: Da die *fastText*-Datei mit den Vektoren (Bezeichnung: *wiki.de.align.vec*) zu groß ist, befindest sich diese nicht im Repository und *Data*. Wenn Sie den Code ausführen möchten, gehen Sie wie folgt vor:
- Laden Sie zunächst den *fastText*-Vektorraum von folgender Seite: https://fasttext.cc/docs/en/aligned-vectors.html (Es handelt sich um den *aligned vector German (Text)*. Direkter Download Link: https://dl.fbaipublicfiles.com/fasttext/vectors-aligned/wiki.de.align.vec [Zuletzt geprüft am 10.01.2022]
- Platzieren Sie die Datei im Repository im Ordner *Data* (oder an einem anderen Ort auf ihrem Rechner und passen Sie dann die Variable *path* entsprechend an.)

In [2]:
def load_vector_data(filepath: str):
    with open(filepath, 'r', encoding='utf-8') as doc:
        next(doc)
        word_vec = {}
        for row in doc:
            row = row.split(' ')
            word = row[0]
            vec = np.array([float(v) for v in row[1:]])
            word_vec[word] = vec
    return word_vec

In [3]:
path = r'..\Data'

word_vec = load_vector_data(join(path, 'wiki.de.align.vec'))
len(word_vec), word_vec['angst'].shape

(2275233, (300,))

### 03 - Das neuronale Netz aufsetzen
Daraus extrahieren wir die Gewichte des Embedding-Layers, die für die unbekannten Wörter initialisiert sind

#### Zunächst die Anzahl der Wörter im Heidegger-Text eruieren

In [5]:
words_in_text = [word for word in set(text_words)]
words_in_text.sort()
words_in_text[:10], len(words_in_text)

(['!', '#', '%', '&', '(', ')', '*', ',', '.', '.a'], 12653)

#### Modell zusammenstellen

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU, Dropout

len_seq = 20
num_words = len(words_in_text)
num_vects = 300 # Länge der Vektoren aus fastText

model = Sequential()
model.add(Embedding(input_dim=num_words, 
                    output_dim=num_vects, 
                    input_length=len_seq))
model.add(GRU(units=num_vects))
model.add(Dense(units=num_words, activation='softmax'))
model.summary()

model.compile(  loss='sparse_categorical_crossentropy', 
                optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 300)           3795900   
_________________________________________________________________
gru_1 (GRU)                  (None, 300)               541800    
_________________________________________________________________
dense_1 (Dense)              (None, 12653)             3808553   
Total params: 8,146,253
Trainable params: 8,146,253
Non-trainable params: 0
_________________________________________________________________


#### Jetzt die Gewichte des Embedding-Layers extrahieren

In [7]:
weights = model.get_layer(index=0).get_weights()[0]
weights.shape

(12653, 300)

### 04 - Den Vektorraum für die Embedding-Schicht aufbauen

In [8]:
def all_word_vectors(words: list, word_vect: dict, weights: np.array ):
    word_vect_index = {}
    word_vect_task = weights.copy()
    count = 0
    for word in words:
        if word in word_vec:
            word_vect_task[count] = word_vect[word]
        word_vect_index[word] = count
        count += 1
    return word_vect_index, word_vect_task

#### Jetzt die Funktion aufrufen. Dabei übergeben wir: 
- die Liste mit den im Text enthaltenen Wörtern (*words_in_test*)
- den *fastText*-Vetkorraum (der die Vektoren aller in *fastText* verzeichneten Wörter enthält, *word_vec*)
- die initialisierten Gewichte aus der Embedding-Layer (*weights*) 

In [10]:
word_vect_index, word_vect_task = all_word_vectors(
                                words_in_text, 
                                word_vec,
                                weights)
len(word_vect_index), word_vect_task.shape

(12653, (12653, 300))

#### Jetzt die Referenz auf das fastText-Objekt kappen, um Platz im Arbeitsspeicher zu schaffen

In [11]:
word_vec = None

### 05 - Den konstruierten Vektorraum in die Embedding-Layer einfügen

In [14]:
model.get_layer(index=0).set_weights([word_vect_task])

### 06 - Die Trainingsdaten vorbereiten

In [15]:
def text_to_sequences( text: list, 
                       words_index: dict, 
                       len_seq: int):
    data_X = []
    data_y = []
    for i in range(len(text)-(len_seq+1)):
        X = text[i:i+len_seq]
        y = text[i+len_seq]
        X = [words_index[word] for word in X]
        y = words_index[y]
        data_X.append(X)
        data_y.append(y)
    return np.array(data_X), np.array(data_y)

In [16]:
X, y = text_to_sequences(text_words, 
                         word_vect_index,
                         len_seq=20)

#### Trainings- und Testdaten separieren

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=12)
X_train.shape, X_test.shape

((157311, 20), (17480, 20))

### 07 - Das Modell anlernen

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint( filepath='heidegger_fasttext_no_missings.h5',
                              save_best_only=False)
stopping = EarlyStopping(monitor='val_loss', patience=1)

history = model.fit(X_train, y_train, 
                    epochs=100, 
                    batch_size=128,
                    validation_data=(X_test, y_test),
                    callbacks=[checkpoint, stopping])

Epoch 1/100
1229/1229 [==============================] - 365s 297ms/step - loss: 6.0979 - accuracy: 0.1080 - val_loss: 5.5061 - val_accuracy: 0.1516
Epoch 2/100
1229/1229 [==============================] - 376s 306ms/step - loss: 5.0342 - accuracy: 0.1758 - val_loss: 5.2738 - val_accuracy: 0.1805
Epoch 3/100
1229/1229 [==============================] - 387s 315ms/step - loss: 4.4170 - accuracy: 0.2143 - val_loss: 5.2504 - val_accuracy: 0.1871
Epoch 4/100
1229/1229 [==============================] - 349s 284ms/step - loss: 3.8226 - accuracy: 0.2589 - val_loss: 5.3415 - val_accuracy: 0.1899


### 08 - Schätzungen durchführen

#### Die Schätzfunktion erstellen

In [19]:
from nltk.tokenize import word_tokenize

def generate_phrases(model, 
                    words_index: dict,
                    starter: str,
                    len_seq = 20, 
                    num_words=100):
    index_words = dict([(idx, keys) for keys, idx in words_index.items()])
    starter = word_tokenize(starter.lower())
    print(' '.join(starter))
    starter = np.array([words_index[word] for word 
                        in starter]).reshape(1, -1)
    for i in range(num_words):
        word_probs = model.predict(starter)[0]
        word_pred = np.argmax(word_probs)
        word = index_words[word_pred]
        print(word, end=' ')
        starter = starter[0].tolist()[1:]
        starter.append(words_index[word])
        starter = np.array(starter).reshape(1, -1)

In [20]:
from tensorflow.keras.models import load_model

model = load_model('heidegger_fasttext_no_missings.h5')

starter = '''wie unterscheidet sich das, wovor die angst sich ängstet, von dem, wovor die furcht sich fürchtet?'''

generate_phrases( model,
                  word_vect_index,  
                  starter,
                  num_words=100 )

wie unterscheidet sich das , wovor die angst sich ängstet , von dem , wovor die furcht sich fürchtet ?
das man ist das gerede , das heißt das , was es ist , ist es , das heißt , was es ist , und zwar , daß es sich in seinem sein um sein sein kann . das sein des seienden ist wesenhaft das sein des daseins . das sein des seienden ist wesenhaft das sein des daseins . das sein des seienden , das wir nennen , ist das sein des daseins . das sein des seienden ist wesenhaft das sein des daseins . das sein des seienden , das wir nennen , ist das sein des daseins 